In [28]:
from gensim.models import Word2Vec
from tensorflow import keras
import tensorflow as tf
from keras.models import load_model
import time
import glob
import os
import os.path
import random
import sys
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import RMSprop

sys.path.append('..')
sys.path.append('../..')

In [29]:
#単語の分散表現のベクトルの次元数
vecSize = 100

#コンテキストのサイズ
contextSize = 20

In [30]:
model = Word2Vec.load("context2Vec.model")

In [31]:
print(len(model.wv.vocab))

print(model.wv.vocab['require'].__dict__)

print(model.wv.vocab)





31546
{'count': 8902, 'index': 53, 'sample_int': 4294967296}
{'require': <gensim.models.keyedvectors.Vocab object at 0x7fdf81c98eb8>, 'on_tstring_beg': <gensim.models.keyedvectors.Vocab object at 0x7fdf81c98390>, 'on_tstring_content': <gensim.models.keyedvectors.Vocab object at 0x7fdf81c98cf8>, 'on_tstring_end': <gensim.models.keyedvectors.Vocab object at 0x7fdf81c98dd8>, 'begin': <gensim.models.keyedvectors.Vocab object at 0x7fdfe15d2518>, 'on_gvar': <gensim.models.keyedvectors.Vocab object at 0x7fdf9050a470>, 'on_op': <gensim.models.keyedvectors.Vocab object at 0x7fdfe1a8f4e0>, 'true': <gensim.models.keyedvectors.Vocab object at 0x7fdfe1a8feb8>, 'rescue': <gensim.models.keyedvectors.Vocab object at 0x7fdfe1a8ff98>, 'loaderror': <gensim.models.keyedvectors.Vocab object at 0x7fdfe1a8fe80>, 'puts': <gensim.models.keyedvectors.Vocab object at 0x7fdfe1a8f710>, 'false': <gensim.models.keyedvectors.Vocab object at 0x7fdfe1a8f908>, 'end': <gensim.models.keyedvectors.Vocab object at 0x7fdfc18

In [32]:
#学習データのファイル群を取得
file = glob.glob("../../repositories_cleansing/repositories2TokenWithCommentDownOnly20Tokens/all.txt")
print(file)

['../../repositories_cleansing/repositories2TokenWithCommentDownOnly20Tokens/all.txt']


In [33]:
context = []
f = open(file[0])
lines = f.readlines()
for line in lines:
    temp_data = line.replace('\n','')
    temp_data = temp_data.split(' ')[0:contextSize]
    context.append(temp_data)
f.close()
print(len(context))

print(context[1])
print(len(context[0]))
print(context[0][0])

48920
['unless', 'on_gvar', 'on_op', 'on_tstring_beg', 'on_tstring_content', 'on_tstring_end', 'UNK', 'on_op', 'UNK', 'on_tstring_content', 'on_tstring_end', 'UNK', 'on_period', 'UNK', 'UNK', 'on_embexpr_beg', 'UNK', 'on_embexpr_end', 'on_tstring_content', 'on_embexpr_beg']
20
require


In [34]:
#テキストのベクトル化
#context2Vec 20トークンに区切った配列を格納している。
context2Vec = []

for i in range(len(context)):
    temp_ary = []
    for j in range(len(context[i])):
        temp_ary.append(np.array(model.wv[context[i][j]]))
    temp_ary = np.array(temp_ary)
    context2Vec.append(temp_ary)
print(context2Vec[0][0])
print(type(context2Vec[0][0]))

[ 2.1514163e+00  2.6678135e+00  1.5311718e-01 -2.6626530e-01
  2.0165567e+00 -2.7255669e+00  1.8864335e+00 -1.0430011e+00
 -3.9171419e+00  2.3399835e+00  1.1281291e+00 -2.5001397e+00
  1.2635750e+00  8.5717666e-01 -7.7713227e-01  8.0980092e-01
  1.1109235e+00  1.4261726e+00  1.2846060e-01  2.1571033e+00
  1.9981530e+00  4.3422309e-01  4.6005696e-01  1.9589333e-01
  7.5064492e-01 -8.9317131e-01  2.0499997e+00 -5.3943002e-01
  6.8521821e-01  1.4595956e+00  1.6101590e+00 -3.1455510e+00
 -4.3687877e-01  8.6277092e-01  5.0332637e+00  6.5039194e-01
 -1.7057325e+00 -1.1438650e+00 -2.8494541e-02  2.2649601e+00
 -6.8490076e-01  1.3662659e+00 -4.3904963e+00  4.8984510e-01
 -3.3512095e-01  5.9127676e-01  6.4703661e-01 -5.0573578e+00
  3.1374874e+00 -2.9515858e+00 -5.8415276e-01  6.0935700e-01
 -1.6160588e+00 -1.9245967e+00  1.5878265e+00  1.1580557e+00
 -1.3117825e+00 -7.2143483e-01  1.8365751e-01  1.8092461e+00
 -3.8733130e+00 -2.1640489e+00  1.2738609e+00  4.9094731e-01
  6.6982394e-01 -2.15571

In [35]:
print(len(context2Vec))

48920


In [36]:
maxLen = contextSize -1

In [37]:
sentences = []
next_chars = []

In [38]:
for i in range(len(context2Vec)):
    sentences.append(context2Vec[i][0:maxLen])
    next_chars.append(context2Vec[i][-1])

In [39]:
print(len(sentences[0]))
print(next_chars[0])

19
[ 3.040895   -1.3832582   0.41968307  0.00953033  0.5529591   1.0076811
  2.819625   -3.0764272  -2.7389262  -0.2954808   2.3148534  -2.624585
  0.48017246  0.5313744  -0.7574382  -1.4951156  -1.5123807  -0.9731355
  1.459493   -0.17908446  1.5912999   2.1466954   3.2414157   0.9480757
  1.5212232   2.9688134   0.67409456  1.5664288   0.01504981  1.223097
  0.674795   -0.11454896  0.200209   -1.8593932   4.0047317  -0.46097958
  0.22197789 -4.3115916   0.14049014  1.6265415   1.7862699  -0.98950994
  0.90461224  0.7206299  -0.96252286 -0.8236698  -1.8490794  -1.5001327
  1.2393072  -3.6131697  -1.7329557   2.567557    1.2900842  -0.37499148
  0.38477623  0.5192246   0.13188092  2.4533596   1.9692652   0.17464745
 -1.5404721  -3.3071578   0.67201656 -1.9877369  -0.72661585  0.9088777
  0.9191363   2.015273    1.7412609  -2.0420027   1.0571761  -3.4406176
  1.1347591   0.55525094 -0.8346183  -0.13913555 -1.4712017  -0.54871947
  1.093732   -0.14739855  1.5928214   0.54782957  2.291691

In [40]:
print(len(sentences))
print(len(sentences[0][0]))

48920
100


In [41]:
# モデルを定義する

In [42]:
models = Sequential()
models.add(LSTM(128,batch_input_shape=(None,maxLen,vecSize)))

In [43]:
models.add(Dense(vecSize))

In [44]:
models.add(Activation('softmax'))

In [45]:
optimizer = RMSprop(lr=0.01)

In [46]:
#損失関数は平均２条誤差を使用

models.compile(loss='mean_squared_error', optimizer=optimizer)

In [47]:
def sample(preds):
    return model.most_similar( [ np.array(preds) ], [], 5)[0][0]

In [48]:
print(model.wv["if"])
model.most_similar([sentences[1][0]],[],1)[0]

[ 3.19403     1.1330805   0.45051655  1.8414989   1.9152901  -1.9906254
  3.347812   -2.872696   -2.1436086  -0.537019    0.36175296 -3.1318712
 -0.8818726   0.75625616  0.45208374  0.27368838 -0.97388536 -0.9457335
  1.6087055   0.86861646  1.4192241   3.072652    3.8982995   0.40674883
  1.1986972   1.1944246   0.34289297 -0.06311839 -0.8335631   1.8101941
  1.9236685  -1.9748133  -0.83260334 -0.19011824  4.975718   -0.8213256
 -3.5922174  -2.5395253  -0.2951625   3.3801935  -0.7993035  -0.17680006
 -4.4002886   2.6482716  -0.9250576  -0.01593615 -1.2022518  -2.6425095
  0.10301345 -1.154618   -2.3255649   1.8915019  -0.4954224  -3.385614
 -1.0041854   0.9965599   1.2260745   1.6338544   0.86889625  1.6845579
 -3.1482768  -3.3478155   0.0442487   1.0369545  -1.4668432  -1.2781453
 -3.4322534   2.740672    1.2113597  -0.75398856 -1.7572935  -4.953053
 -0.2081256  -0.53858143  0.46458983 -0.67906857 -1.5375513  -1.6469752
  3.1392202   1.6463885   0.17616947  1.1431563   1.2104276   0.

/Users/makoto/anaconda3/envs/ruby_rnn/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


('unless', 1.0)

In [49]:
for iteration in range(1,30):
    print()
    print('-'*50)
    print('繰り返し回数: ', iteration)
    models.fit(np.array(sentences),np.array(next_chars),epochs=2)

    start_index = random.randint(0, len(sentences)-1)
    while model.most_similar([sentences[start_index][0] ],[],1)[0][0] == "UNK":
        start_index = random.randint(0, len(sentences))

    generated = ''
    #ベクトル表現から単語を得ている。
    #初期値となる単語を得ている。
    sentence = model.most_similar([sentences[start_index][0] ],[],1)[0][0]
    generated += sentence
    print('-----Seedを生成しました"' + sentence + '"')

    sys.stdout.write(generated)

    x = sentences[start_index]
    for i in range(400):
        input_x = x[-maxLen:]
        input_x = np.array(input_x)
        #形を(1,maxLen,textSize)にするため
        input_x = input_x.reshape(1,input_x.shape[0],input_x.shape[1])
        #次の単語の分散表現を得ている。
        preds = models.predict(input_x,verbose=2)[0]
        
        next_char = sample(preds)
        generated += next_char
        generated += " "
        x = np.append(x, np.array([model.wv[next_char]]),axis=0)
    print()
    print(generated)
    print()
    


--------------------------------------------------
繰り返し回数:  1
Epoch 1/2
1529/1529 [==============================] - 13s 9ms/step - loss: 1.9724
Epoch 2/2
1529/1529 [==============================] - 14s 9ms/step - loss: 1.9714
-----Seedを生成しました"def"
def

/Users/makoto/anaconda3/envs/ruby_rnn/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/Users/makoto/anaconda3/envs/ruby_rnn/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s


/Users/makoto/anaconda3/envs/ruby_rnn/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1

...

Epoch 1/2
1529/1529 [==============================] - 18s 12ms/step - loss: 1.9699
Epoch 2/2
1529/1529 [==============================] - 19s 12ms/step - loss: 1.9698
-----Seedを生成しました"true"
true1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s

defcompiler compiler cmdline_debug compiler UNK compiler cmdline_debug compiler tmp_file compiler UNK compiler read_json_attribute UNK compiler compiler UNK compiler info compiler compiler info UNK on_rbracket UNK info UNK info compiler eval_after on_tstring_content compiler eval_after on_tstring_content compiler read_json_attribute info UNK separator info UNK separator info UNK separator info UNK UNK UNK compiler info UNK on_rbracket UNK info compiler on_tstring_beg on_tstring_content compiler on_ivar on_tstring_beg on_tstring_content compiler on_ivar read_json_attribute on_ivar UNK eval_after info eval_after info UNK info UNK on_ivar UNK eval_after info compiler UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK compiler on_ivar info UNK info UNK on_ivar on_tstring_beg on_tstring_co

Epoch 1/2
1529/1529 [==============================] - 22s 15ms/step - loss: 1.9699
Epoch 2/2
1529/1529 [==============================] - 22s 14ms/step - loss: 1.9699
-----Seedを生成しました"result"
result1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s


1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1

In [50]:
models.save("context2Seq.model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


FailedPreconditionError: context2Seq.model is not a directory

In [ ]:
#ここからはmodelをloadしてきて遊んでみる


In [ ]:
models = keras.models.load_model("context2Seq.model")
print(np.array([context2Vec[0][:maxLen]]).shape)

In [ ]:
preds = models.predict(np.array([context2Vec[1][:maxLen]]),verbose=2)

In [ ]:
preds[0]